# more evaluatations for the joint model

First seeing can we trian a model with masking rate of 0, what does it do?

In [2]:
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus/')
from src.dataloaders.datasets.general_dataset import GeneralDataset
dataset = GeneralDataset(
    split='train',
    preprocess=False,
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz',
    length=524288,
    shift_sequences=1000,
    load_in=False,
    mlm=.25,
    acc_mlm=.25,
    acc_type='continuous',
    acc_mask_size=500,
)

In [3]:
#let's see an example
out = dataset[0]

In [4]:
out[1][0].shape

torch.Size([524288, 6])

In [5]:
import torch
random_pred = torch.rand(out[1][0][:,:-1].shape)
random_pred.shape

torch.Size([524288, 5])

In [6]:
out[1][0]

tensor([[0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 1.]])

In [7]:
random_pred.unsqueeze(0).shape, out[1][0].unsqueeze(0).shape

(torch.Size([1, 524288, 5]), torch.Size([1, 524288, 6]))

In [8]:
#and then we compute the loss
import torch.nn.functional as F
def ce_loss_mask_seq(x, y):
    """
    Cross entropy loss for sequence classification.
    
    x: tuple (seq, dummy)
         - seq: (batch_size, seq_len, vocab_size)
    y: tuple (seq_unmask, dummy)
         - seq_unmask: (batch_size, seq_len, vocab_size+1)  (last channel is the mask)
    """
    seq = x[0]
    seq_unmask = y[0]
    
    # Create mask from last column of seq_unmask
    mask = seq_unmask[:, :, -1] == 1
    seq_pred = seq[mask]
    # Remove mask channel from target; resulting shape is (N, vocab_size)
    seq_target = seq_unmask[mask][:, :-1]
    
    loss = F.cross_entropy(seq_pred, seq_target)
    return loss
ce_loss_mask_seq((random_pred.unsqueeze(0),None), (out[1][0].unsqueeze(0),None))

tensor(1.6429)

In [15]:
#see that works, what if mask returns nothing
dataset = GeneralDataset(
    split='train',
    preprocess=False,
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz',
    length=524288,
    shift_sequences=1000,
    load_in=False,
    mlm=0,
    acc_mlm=.25,
    acc_type='continuous',
    acc_mask_size=500,
)
out = dataset[0]

RuntimeError: "bernoulli_tensor_cpu_p_" not implemented for 'Long'

In [16]:
#let's see what it's doing and what the issue is
chrom, start, end, split = dataset.sequences.iloc[0]
seq = dataset.genome[chrom][start:end]
seq.shape

(524288,)

In [17]:
seq = torch.LongTensor(seq)

In [18]:
x = seq
x_onehot = torch.nn.functional.one_hot(x-7, num_classes=5).float() #N is its own class, also no transpose, so shape is seq_lenx5
seq = x_onehot
seq.shape

torch.Size([524288, 5])

In [19]:
span=1
mask_pct = 0.15
num_elements = seq.shape[0]//span
probability_matrix = torch.full((num_elements,), mask_pct)
masked_indices = torch.bernoulli(probability_matrix).bool()

In [20]:
span=1
mask_pct = 0
num_elements = seq.shape[0]//span
probability_matrix = torch.full((num_elements,), mask_pct)
masked_indices = torch.bernoulli(probability_matrix).bool()

RuntimeError: "bernoulli_tensor_cpu_p_" not implemented for 'Long'

In [23]:
probability_matrix.dtype

torch.int64

In [24]:
span=1
mask_pct = 0
num_elements = seq.shape[0]//span
probability_matrix = torch.full((num_elements,), mask_pct).float()
masked_indices = torch.bernoulli(probability_matrix).bool()

In [25]:
span=1
mask_pct = 0.15
num_elements = seq.shape[0]//span
probability_matrix = torch.full((num_elements,), mask_pct).float()
masked_indices = torch.bernoulli(probability_matrix).bool()

In [ ]:
random_pred.unsqueeze(0).shape, out[1][0].unsqueeze(0).shape

(torch.Size([1, 524288, 5]), torch.Size([1, 524288, 6]))

In [1]:
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus/')
from src.dataloaders.datasets.general_dataset import GeneralDataset
#and then we compute the loss
#see that works, what if mask returns nothing
dataset = GeneralDataset(
    split='train',
    preprocess=False,
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/cell_type_arrays/GM12878_DNase.npz',
    length=524288,
    shift_sequences=1000,
    load_in=False,
    mlm=0,
    acc_mlm=.25,
    acc_type='continuous',
    acc_mask_size=500,
)
out = dataset[0]

In [2]:
out[1][0].shape

torch.Size([524288, 6])

In [3]:
import torch
random_pred = torch.rand(out[1][0][:,:-1].shape)
random_pred.shape

torch.Size([524288, 5])

In [ ]:
out[1][0].sum(0) #no masks, this is fine!

tensor([124427.,  90194.,  88400., 110544., 110723.,      0.])

In [12]:
#let's 

import torch.nn.functional as F
def ce_loss_mask_seq(x, y):
    """
    Cross entropy loss for sequence classification. Note that if nothing is masked, it checks everything
    
    x: tuple (seq, dummy)
         - seq: (batch_size, seq_len, vocab_size)
    y: tuple (seq_unmask, dummy)
         - seq_unmask: (batch_size, seq_len, vocab_size+1)  (last channel is the mask)
    """
    seq = x[0]
    seq_unmask = y[0]
    
    # Create mask from last column of seq_unmask
    mask = seq_unmask[:, :, -1] == 1
    
    seq_pred = seq[mask]
    # Remove mask channel from target; resulting shape is (N, vocab_size)
    seq_target = seq_unmask[mask][:, :-1]
    
    loss = F.cross_entropy(seq_pred, seq_target)
    return loss
ce_loss_mask_seq((random_pred.unsqueeze(0),None), (out[1][0].unsqueeze(0),None))

tensor(nan)

In [10]:
mask  = out[1][0][:,-1] == 1
mask.sum()

tensor(0)

In [11]:
mask.sum().item()

0

In [13]:
def ce_loss_mask_seq(x, y):
    """
    Cross entropy loss for sequence classification. Note that if nothing is masked, it checks everything
    
    x: tuple (seq, dummy)
         - seq: (batch_size, seq_len, vocab_size)
    y: tuple (seq_unmask, dummy)
         - seq_unmask: (batch_size, seq_len, vocab_size+1)  (last channel is the mask)
    """
    seq = x[0]
    seq_unmask = y[0]
    
    # Create mask from last column of seq_unmask
    mask = seq_unmask[:, :, -1] == 1
    
    if mask.sum().item() == 0:
        # If no mask is present, just calculate it on everything
        mask = torch.ones_like(seq_unmask[:, :, -1], dtype=torch.bool)
        
    seq_pred = seq[mask]
    # Remove mask channel from target; resulting shape is (N, vocab_size)
    seq_target = seq_unmask[mask][:, :-1]
    
    loss = F.cross_entropy(seq_pred, seq_target)
    return loss
ce_loss_mask_seq((random_pred.unsqueeze(0),None), (out[1][0].unsqueeze(0),None))

tensor(1.6422)

In [14]:
#and let's see if we do class indices if it's more efficient
def ce_loss_mask_seq(x, y):
    """
    Cross entropy loss for sequence classification. Note that if nothing is masked, it checks everything
    
    x: tuple (seq, dummy)
         - seq: (batch_size, seq_len, vocab_size)
    y: tuple (seq_unmask, dummy)
         - seq_unmask: (batch_size, seq_len, vocab_size+1)  (last channel is the mask)
    """
    seq = x[0]
    seq_unmask = y[0]
    
    # Create mask from last column of seq_unmask
    mask = seq_unmask[:, :, -1] == 1
    
    if mask.sum().item() == 0:
        # If no mask is present, just calculate it on everything
        mask = torch.ones_like(seq_unmask[:, :, -1], dtype=torch.bool)
        
    seq_pred = seq[mask]
    # Remove mask channel from target; resulting shape is (N, vocab_size)
    seq_target = seq_unmask[mask][:, :-1]
    
    loss = F.cross_entropy(seq_pred, seq_target.argmax(dim=-1))
    return loss
ce_loss_mask_seq((random_pred.unsqueeze(0),None), (out[1][0].unsqueeze(0),None))

tensor(1.6422)

In [17]:
seq_unmask = out[1][0].unsqueeze(0)
mask = torch.ones_like(seq_unmask[:, :, -1], dtype=torch.bool)
seq_target = seq_unmask[mask][:, :-1]
seq_target.shape

torch.Size([524288, 5])

In [18]:
seq_target

tensor([[0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        ...,
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
seq_target.argmax(dim=-1) #yeah works as expecteed!!

tensor([4, 4, 4,  ..., 3, 3, 1])

In [27]:
import time
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, seq_target.argmax(dim=-1))
print(f'Time taken for indices: {time.time()-start} seconds')
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, seq_target)
print(f'Time taken for one hot: {time.time()-start} seconds')

Time taken for indices: 5.562175750732422 seconds
Time taken for one hot: 3.487555980682373 seconds


In [28]:
#let's test this on the GPU, we've been doing it on the cpu
random_pred = random_pred.cuda()
seq_target = seq_target.cuda()
start = time.time() 
for i in range(1000):
    F.cross_entropy(random_pred, seq_target.argmax(dim=-1))
print(f'Time taken for indices: {time.time()-start} seconds')
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, seq_target)
print(f'Time taken for one hot: {time.time()-start} seconds')

Time taken for indices: 6.6588006019592285 seconds
Time taken for one hot: 3.4974451065063477 seconds


In [35]:
#let's test it more rigorously, set a torch random seed
torch.manual_seed(0)
vals_indices = []
start = time.time()
for i in range(1000):
    random_pred = torch.rand(out[1][0][:,:-1].shape).cuda()
    seq_target = seq_target.cuda()
    vals_indices.append(F.cross_entropy(random_pred, seq_target.argmax(dim=-1)))
print(f'Time taken for indices: {time.time()-start} seconds')

Time taken for indices: 14.083431959152222 seconds


In [36]:
torch.manual_seed(0)
vals_onehot = []
start = time.time()
for i in range(1000):
    random_pred = torch.rand(out[1][0][:,:-1].shape).cuda()
    seq_target = seq_target.cuda()
    vals_onehot.append(F.cross_entropy(random_pred, seq_target))
print(f'Time taken for one hot: {time.time()-start} seconds')

Time taken for one hot: 12.698261737823486 seconds


In [42]:
torch.allclose(torch.tensor(vals_indices),torch.tensor(vals_onehot))

True

In [43]:
random_pred.shape

torch.Size([524288, 5])

In [47]:
random_pred = torch.rand((32000,5)).cuda()
random_target = torch.randint(0,5,(32000,)).cuda()
random_target_onehot = torch.nn.functional.one_hot(random_target, num_classes=5).float()
print(random_target)
print(random_target_onehot)

tensor([1, 2, 2,  ..., 3, 4, 1], device='cuda:0')
tensor([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        ...,
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]], device='cuda:0')


In [48]:
random_target_onehot.shape, random_target.shape

(torch.Size([32000, 5]), torch.Size([32000]))

In [49]:
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, random_target_onehot)
print(f'Time taken for one hot: {time.time()-start} seconds')

Time taken for one hot: 0.03280234336853027 seconds


In [51]:
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, random_target)
print(f'Time taken for indices: {time.time()-start} seconds')

Time taken for indices: 0.2142198085784912 seconds


In [ ]:
start = time.time()
for i in range(1000):
    F.cross_entropy(random_pred, random_target_onehot.argmax(dim=-1))
print(f'Time taken for indices: {time.time()-start} seconds')
#why is this the slowest, I don't get it? let's stick to one hot labels I guess

Time taken for indices: 0.31087613105773926 seconds
